In [ ]:
!pip install langchain[all] chromadb
!pip install configparser

In [4]:
import configparser
api_config = configparser.ConfigParser()
api_config.read_file(open('apidata.config'))

In [5]:
import os
os.environ['OPENAI_API_KEY'] = api_config["OPENAI"]["KEY"]

In [6]:
from langchain.agents import create_pandas_dataframe_agent

from langchain.llms import OpenAI

import pandas as pd

In [8]:
spacedf = pd.read_csv('/content/space_shortened.csv')

In [9]:
fact_llm = OpenAI(temperature=0)

spagent = create_pandas_dataframe_agent(fact_llm, spacedf,verbose=True)

In [ ]:
spagent.run("How many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 16
Thought: I now know the final answer
Final Answer: 16

> Finished chain.


'16'

In [ ]:
spagent.run("Split the Name column into First and last name, which lastname has highest frequency?")



> Entering new AgentExecutor chain...
Thought: I need to split the Name column and then count the frequency of each lastname
Action: python_repl_ast
Action Input: df['Name'].str.split(' ', expand=True).rename(columns={0:'FirstName', 1:'LastName'}).groupby('LastName').size().sort_values(ascending=False).head(1)
Observation: LastName
Flatic    3
dtype: int64
Thought: I now know the final answer
Final Answer: Flatic

> Finished chain.


'Flatic'

In [ ]:
name_df = spacedf['Name'].str.split(' ', expand=True).rename(columns={0:'FirstName', 1:'LastName'})

In [ ]:
name_df['LastName'].value_counts()

Flatic         3
Susent         2
Jacostaffey    2
Ofracculy      1
Vines          1
Santantines    1
Hinetthews     1
Beston         1
Barne          1
Baketton       1
Bertsontry     1
Pooles         1
Name: LastName, dtype: int64

In [ ]:
spagent.run("Summarize the data into 50 words")



> Entering new AgentExecutor chain...
Thought: I need to look at the data and summarize it
Action: python_repl_ast
Action Input: df.describe()
Observation:              Age  RoomService   FoodCourt  ShoppingMall          Spa  \
count  16.000000    16.000000    16.00000     15.000000    16.000000   
mean   33.437500    85.500000  1000.93750    215.466667   736.062500   
std    11.512131   185.335372  1928.60172    363.000564  1793.858503   
min    14.000000     0.000000     0.00000      0.000000     0.000000   
25%    27.500000     0.000000     0.00000      0.000000     0.000000   
50%    32.500000    20.000000    39.50000     17.000000     1.000000   
75%    40.250000    50.500000  1051.25000    261.000000   355.500000   
max    58.000000   719.000000  7295.00000   1123.000000  6715.000000   

           VRDeck  
count   15.000000  
mean    37.066667  
std     59.612399  
min      0.000000  
25%      0.000000  
50%      2.000000  
75%     46.500000  
max    193.000000  
Thought: I no

'The dataframe contains 16 passengers with information about their home planet, cryosleep status, cabin, destination, age, VIP status, room service, food court, shopping mall, spa, and VR deck. The average age of passengers is 33.4, with a standard deviation of 11.5. Room service, food court, shopping mall, spa, and VR deck all have a wide range of values, with the highest being 7295, 6715, 1123, and 193 respectively.'

In [10]:
from langchain.agents import create_json_agent

from langchain.agents.agent_toolkits import JsonToolkit

from langchain.tools.json.tool import JsonSpec

import json

In [ ]:


with open('/content/impressions.json') as imp:
  data = json.load(imp)

In [ ]:
with open('/content/impressions.json') as chek:
  print(len(chek.read()))

320900


In [ ]:
data.keys()

dict_keys(['impressions'])

In [ ]:
data['impressions'][0].keys()

dict_keys(['impid', 'aid', 'name'])

In [ ]:
#data used to create jsonspec
imp_json_spec = JsonSpec(dict_=data, max_value_length=4000)
#toolkit uses the spec
imp_json_toolkit = JsonToolkit(spec=imp_json_spec)
#agent
imp_agent = create_json_agent(
    llm=OpenAI(temperature=0),
    toolkit=imp_json_toolkit,
    verbose=True
)

In [ ]:
query1 = "How many impressions are there?"

imp_agent.run(query1)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['impressions']
Thought: I should use json_spec_get_value to get the value of impressions
Action: json_spec_get_value
Action Input: data["impressions"]
Observation: [{'impid': 4142, 'aid': 'Chris Redd', 'name': 'Barack Obama'}, {'impid': 4143, 'aid': 'Beck Bennett', 'name': 'Bruce Springsteen'}, {'impid': 4141, 'aid': 'Pete Davidson', 'name': 'Matt Gaetz'}, {'impid': 4057, 'aid': 'Chloe Fineman', 'name': 'Britney Spears'}, {'impid': 4140, 'aid': 'Chris Redd', 'name': 'Lil Nas X'}, {'impid': 2258, 'aid': 'Maya Rudolph', 'name': 'Beyoncé'}, {'impid': 4134, 'aid': 'Mikey Day', 'name': 'Sean Evans'}, {'impid': 4005, 'aid': 'Maya Rudolph', 'name': 'Kamala Harris'}, {'impid': 4135, 'aid': 'Martin Short', 'name': 'Doug Emhoff'}, {'impid': 4121, 'aid': 'Aidy Bryant', 'name': 'Ted Cruz'}, {'impid': 4136, 'aid': 'Chloe Fineman', 'name': 'Ella Emhoff'}, {'impid': 4096, 'aid': 'Alex Moffat', 'name':

'There are 43 impressions.'

In [ ]:
query2 = "What is the range of Impid?"
imp_agent.run(query2)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['impressions']
Thought: I should look at the keys that exist in impressions to see what I have access to
Action: json_spec_list_keys
Action Input: data["impressions"][0]
Observation: ValueError('Value at path `data["impressions"][0]` is not a dict, get the value directly.')
Thought: I should get the value at the path to see what I have access to
Action: json_spec_get_value
Action Input: data["impressions"][0]
Observation: {'impid': 4142, 'aid': 'Chris Redd', 'name': 'Barack Obama'}
Thought: I should look at the keys that exist in the value to see what I have access to
Action: json_spec_list_keys
Action Input: data["impressions"][0]
Observation: ValueError('Value at path `data["impressions"][0]` is not a dict, get the value directly.')
Thought: I should get the value at the path to see what I have access to
Action: json_spec_get_value
Action Input: data["impressions"][0]["impid"]
Observa

'The range of Impid is 4142.'

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    result = fact_llm("Tell me a joke")
    print(cb.total_tokens)

24


In [ ]:
with get_openai_callback() as cb:
  query3 = "What names are available in aid?"
  imp_agent.run(query3)
  print(cb.total_tokens)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['impressions']
Thought: I should look at the keys inside impressions
Action: json_spec_list_keys
Action Input: data["impressions"]
Observation: ValueError('Value at path `data["impressions"]` is not a dict, get the value directly.')
Thought: I should get the value of impressions
Action: json_spec_get_value
Action Input: data["impressions"]
Observation: [{'impid': 4142, 'aid': 'Chris Redd', 'name': 'Barack Obama'}, {'impid': 4143, 'aid': 'Beck Bennett', 'name': 'Bruce Springsteen'}, {'impid': 4141, 'aid': 'Pete Davidson', 'name': 'Matt Gaetz'}, {'impid': 4057, 'aid': 'Chloe Fineman', 'name': 'Britney Spears'}, {'impid': 4140, 'aid': 'Chris Redd', 'name': 'Lil Nas X'}, {'impid': 2258, 'aid': 'Maya Rudolph', 'name': 'Beyoncé'}, {'impid': 4134, 'aid': 'Mikey Day', 'name': 'Sean Evans'}, {'impid': 4005, 'aid': 'Maya Rudolph', 'name': 'Kamala Harris'}, {'impid': 4135, 'aid': 'Martin Short', '

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
hugllm=HuggingFaceHub(repo_id="google/flan-t5-xl", 
                   model_kwargs={"temperature":0,
                                 "max_length":64})

In [ ]:
llm_chain = LLMChain(prompt=prompt,llm=hugllm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

print(llm_chain.run(question))

The Seattle Seahawks won the Super Bowl in 2010. Justin Beiber was born in 2010. The final answer: Seattle Seahawks.


In [ ]:
sp_hug_agent = create_pandas_dataframe_agent(hugllm, 
                                        spacedf,
                                        verbose=True)

In [ ]:
sp_hug_agent.run("How many rows are there?")

In [ ]:
print(llm_chain.run("Help in resolving the error. ValueError: Could not parse LLM output: Answer: 0"))

The LLM file is not valid. The LLM file is not a valid XML file. The XML file is not a valid XML file. The XML file is not a valid XML file. The XML file is not a valid 


OpenAPI spec using the Language model

In [11]:
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.requests import RequestsWrapper

In [12]:
with open("/content/pyrite_openapi.json") as f:
    pyrite = json.load(f)
json_spec=JsonSpec(dict_=pyrite, max_value_length=4000)

In [13]:
requests_wrapper=RequestsWrapper()
openapi_toolkit = OpenAPIToolkit.from_llm(OpenAI(temperature=0), 
                                          json_spec, 
                                          requests_wrapper, 
                                          verbose=True)

In [14]:
openapi_agent_executor = create_openapi_agent(
    llm=OpenAI(temperature=0),
    toolkit=openapi_toolkit,
    verbose=True
)

In [16]:
openapi_agent_executor.run("Make a get requests to pyrite-ethereal-soccer.glitch.me /deliveryd endpoint")



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'paths', 'components']
Thought: I should look at the info key to see if I can find the base url
Action: json_spec_get_value
Action Input: data["info"]["baseUrl"]
Observation: KeyError('baseUrl')
Thought: I should look at the info key to see if I can find the basePath
Action: json_spec_get_value
Action Input: data["info"]["basePath"]
Observation: KeyError('basePath')
Thought: I should look at the paths key to see if I can find the base url
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/user/create_user', '/user/v1/show_users', '/user/id/{user_id}', '/user/login', '/user/validate', '/biker/v1/bikers', '/biker/v1/deliveryd', '/biker/v1/pickupd', '/biker/v1/features', '/biker/v1/predict', '/']
Thought: I now know the final answer
Final Answ

MissingSchema: ignored

In [17]:
openapi_agent_executor.run("Make a post requests to pyrite-ethereal-soccer.glitch.me and create user largeLLM with password 77tro*the&*&")



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'paths', 'components']
Thought: I should look at the info key to see if I can find the base url
Action: json_spec_get_value
Action Input: data["info"]["baseUrl"]
Observation: KeyError('baseUrl')
Thought: I should look at the paths key to see if I can find the base url
Action: json_spec_get_value
Action Input: data["paths"]["baseUrl"]
Observation: KeyError('baseUrl')
Thought: I should look at the components key to see if I can find the base url
Action: json_spec_get_value
Action Input: data["components"]["baseUrl"]
Observation: KeyError('baseUrl')
Thought: I don't know the answer to this question
Final Answer: I don't know

> Finished chain.

Observation: I don't know
Thought: I should look for the paths and parameters needed to make the request.
Action: json_ex

'The user largeLLM was successfully created.'

In [18]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

In [19]:
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=500),
    tool=PythonREPLTool(),
    verbose=True
)

In [20]:
agent_executor.run("Write a program that automatically converts English text to Morse code and vice versa.")



> Entering new AgentExecutor chain...
 I need to create a dictionary that maps English letters to Morse code and vice versa.
Action: Python REPL
Action Input: 
morse_code = {
    'A': '.-', 'B': '-...',
    'C': '-.-.', 'D': '-..', 'E': '.',
    'F': '..-.', 'G': '--.', 'H': '....',
    'I': '..', 'J': '.---', 'K': '-.-',
    'L': '.-..', 'M': '--', 'N': '-.',
    'O': '---', 'P': '.--.', 'Q': '--.-',
    'R': '.-.', 'S': '...', 'T': '-',
    'U': '..-', 'V': '...-', 'W': '.--',
    'X': '-..-', 'Y': '-.--', 'Z': '--..',
    '1': '.----', '2': '..---', '3': '...--',
    '4': '....-', '5': '.....', '6': '-....',
    '7': '--...', '8': '---..', '9': '----.',
    '0': '-----', ', ': '--..--', '.': '.-.-.-',
    '?': '..--..', '/': '-..-.', '-': '-....-',
    '(': '-.--.', ')': '-.--.-'
}

Observation: 
Thought: I now need to create a function that takes in a string and converts it to Morse code or vice versa.
Action: Python REPL
Action Input: 
def convert_to_morse(text):
    morse_text 

'I have written a program that automatically converts English text to Morse code and vice versa.'

In [21]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor

In [24]:
db = SQLDatabase.from_uri("sqlite:////content/chinook.db")
toolkit = SQLDatabaseToolkit(db=db)

In [25]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)

In [26]:
agent_executor.run("List the tables inside the database")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Album, Genre, Artist, MediaType
Thought: I now know the tables in the database
Final Answer: The tables in the database are Album, Genre, Artist, and MediaType.

> Finished chain.


'The tables in the database are Album, Genre, Artist, and MediaType.'

In [27]:
agent_executor.run("What are the details inside the artist table")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Album, Genre, Artist, MediaType
Thought: I should look at the schema of the artist table
Action: schema_sql_db
Action Input: "Artist"
Observation: 
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)
/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Thought: I now know the details inside the artist table
Final Answer: The Artist table contains two columns, ArtistId and Name, and contains information about artists such as AC/DC, Accept, and Aerosmith.

> Finished chain.


'The Artist table contains two columns, ArtistId and Name, and contains information about artists such as AC/DC, Accept, and Aerosmith.'

In [29]:
agent_executor.run("Which artist has most albums")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Album, Genre, Artist, MediaType
Thought: I should check the schema of the tables to see what columns I can query.
Action: schema_sql_db
Action Input: "Album, Artist"
Observation: 
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)
/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)
/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/
Thought: I should query the Album table to get the artist with the most albums.
Action: query_sql_db
Action Input: SELECT Artist.Name, COUNT(Album.AlbumId) AS AlbumCount FROM Album INNER JOIN Ar

'Iron Maiden has the most albums with 21.'